# 普通稀疏卷积

In [1]:
# !pip install spconv-cu113

- https://github.com/traveller59/spconv
- https://github.com/traveller59/spconv/blob/master/docs/USAGE.md

In [2]:
import spconv.pytorch as spconv
import torch 

In [3]:
# !export CUDA_VISIBLE_DEVICES=1


- 想象成，经过体素化后，3D世界被划分为4*4*1的格子
- 有的格子是没有特征的。有的格子特征features分别为[1] [2] [3]
- 索引是[0, 0, 0], [1, 1, 0], [3, 3, 0]

In [4]:
features = torch.tensor([[1], [2], [3]])# your features with shape [N, num_channels]
indices = torch.tensor([[0, 0, 0, 0], [0, 1, 1, 0], [0, 3, 3, 0]], dtype=torch.int32)# your indices/coordinates with shape [N, ndim + 1], batch index must be put in indices[:, 0]
# 第一个位置是batch_idx，后四位是features对应的索引值
spatial_shape = torch.tensor([4, 4, 1])# spatial shape of your sparse tensor, spatial_shape[i] is shape of indices[:, 1 + i].
batch_size = 1# batch size of your sparse tensor.
x = spconv.SparseConvTensor(features, indices, spatial_shape, batch_size)
print(x)
print(x.features)
x_dense_NCHW = x.dense() # convert sparse tensor to dense NCHW tensor.
print(x_dense_NCHW.shape)
print(x_dense_NCHW.dtype)
print(x_dense_NCHW)

SparseConvTensor[shape=torch.Size([3, 1])]
tensor([[1],
        [2],
        [3]])
torch.Size([1, 1, 4, 4, 1])
torch.int64
tensor([[[[[1],
           [0],
           [0],
           [0]],

          [[0],
           [2],
           [0],
           [0]],

          [[0],
           [0],
           [0],
           [0]],

          [[0],
           [0],
           [0],
           [3]]]]])


In [5]:
import spconv.pytorch as spconv
from torch import nn
class ExampleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = spconv.SparseSequential(
            spconv.SparseConv3d(1, 1, 3, 1, 1, bias=False), 
        )

    def forward(self, sparse_input):
        x_sp = spconv.SparseConvTensor.from_dense(sparse_input.reshape(-1, 1, 4, 4, 1))
        return self.net(x_sp)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = ExampleNet()
model.net[0].weight.data.fill_(1)
model.to(device)

x_dense_NCHW = x_dense_NCHW.to(torch.float32).to(device) # 必须转类型

output = model(x_dense_NCHW)
output.dense()

tensor([[[[[3., 3., 2., 0.],
           [3., 3., 2., 0.],
           [2., 2., 5., 3.],
           [0., 0., 3., 3.]]]]], device='cuda:0', grad_fn=<PermuteBackward0>)

# 二、SubM卷积

In [8]:
import spconv.pytorch as spconv
from torch import nn
class ExampleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = spconv.SparseSequential(
            spconv.SubMConv3d(1, 1, 3, 1, 1, bias=False), 
        )

    def forward(self, sparse_input):
        x_sp = spconv.SparseConvTensor.from_dense(sparse_input.reshape(-1, 1, 4, 4, 1))
        return self.net(x_sp)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ExampleNet()
model.net[0].weight.data.fill_(1)
model.to(device)

x_dense_NCHW = x_dense_NCHW.to(torch.float32).to(device) # 必须转类型

output = model(x_dense_NCHW)
output.dense()

tensor([[[[[3., 0., 0., 0.],
           [0., 3., 0., 0.],
           [0., 0., 0., 0.],
           [0., 0., 0., 3.]]]]], device='cuda:0', grad_fn=<PermuteBackward0>)

- 希望这个案例，能够让大家知道，点云体素化后，
    - 有了features 和 coors ，coors通常是batch_size，X,Y,Z

- 使得我们虽然没有细看稀疏卷积网络，但是能对他有个大概的印象。

# 其他，不要看

In [10]:
import torch.nn as nn
import torch

In [11]:
a = nn.Linear.__new__(nn.Linear)
b = nn.Linear(1, 3)

In [12]:
i = 0
for k, v in vars(b).items():
    i+=1
    setattr(a, k, v)
    if i == 11:
        print("1")
a

1


Linear(in_features=1, out_features=3, bias=True)

In [13]:
setattr(a, "in_features", 1)
setattr(a, "out_features", 3)
setattr(a, "bias", torch.tensor(3))
# weight = torch.ones((3, 1))
# print(weight.shape)
setattr(a, "weight", torch.tensor([[1., 2., 3.]]).T)
setattr(a, "device", "cuda:0")
setattr(a, "dtype", torch.float32)
setattr(a, "aaa", 3)

print(getattr(a, "in_features"))
print(getattr(a, "out_features"))
print(getattr(a, "bias"))
print(getattr(a, "weight"))

intput = torch.ones((1))
print(intput.shape)
res = a.forward(intput)
res

TypeError: cannot assign 'torch.LongTensor' as parameter 'bias' (torch.nn.Parameter or None expected)